In [1]:
from pandas import DataFrame, Series
import pandas as pd
import aif360
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
# read in the main training csv
train_main = pd.read_csv('home-credit-default-risk/application_train.csv', na_values=['','XNA','XAP'])

In [5]:
print(train_main.shape)
train_main[:5]

(307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [6]:
# Drop records that are too sparse (after previous discovery, these are usually not represented in val and/or test data
# Family Status: Unknown
train_main = train_main[train_main['NAME_FAMILY_STATUS'] != 'Unknown'] #2 records with unknown family status
train_main = train_main[train_main['NAME_INCOME_TYPE'] != 'Businessman'] #10 businessman
train_main = train_main[train_main['NAME_INCOME_TYPE'] != 'Maternity leave'] # 5 maternity leave

In [7]:
train_main.shape # 17 records dropped

(307494, 122)

### Binarize the protected attributes - CODE_GENDER, DAYS_BIRTH, NAME_FAMILY_STATUS - fairness

In [8]:
print(train_main['CODE_GENDER'].value_counts())
train_main.groupby(['CODE_GENDER'])['TARGET'].mean()
# unprivileged_group: male (who get rejected more)

F    202440
M    105050
Name: CODE_GENDER, dtype: int64


CODE_GENDER
F    0.069986
M    0.101428
Name: TARGET, dtype: float64

In [9]:
def ismale(gender):
    """return 1 if male; 0 if female"""
    if gender == 'M':
        return 1
    else:
        return 0

train_main['IsMale'] = train_main['CODE_GENDER'].map(ismale)
train_main['IsMale'].value_counts() # a few nas got coded as female
# the treatment for those nas was going to be filled with female anyway, so good

0    202444
1    105050
Name: IsMale, dtype: int64

In [10]:
print(train_main['NAME_FAMILY_STATUS'].value_counts())
train_main.groupby(['NAME_FAMILY_STATUS'])['TARGET'].mean()
# married people seem to get approved more (esp. given it's higher absolute numbers)
# unprivileged_groups: single
# set up: married, separated, civil = 0; single/widow = 1

Married                 196420
Single / not married     45442
Civil marriage           29774
Separated                19770
Widow                    16088
Name: NAME_FAMILY_STATUS, dtype: int64


NAME_FAMILY_STATUS
Civil marriage          0.099449
Married                 0.075593
Separated               0.081942
Single / not married    0.098081
Widow                   0.058242
Name: TARGET, dtype: float64

In [11]:
# Binarize family status
def bifam(status):
    """
    Determine if an applicant has ever been married
    arg: status of applicant
    return: 1 if single / Widow; 0 if otherwise
    """
    if status in ["Single / not married", "Widow"]:
        return 1
    else:
        return 0

train_main['MaritalSingle'] = train_main['NAME_FAMILY_STATUS'].map(bifam)
train_main['MaritalSingle'].value_counts()

0    245964
1     61530
Name: MaritalSingle, dtype: int64

In [12]:
print("Applicants range from", -train_main['DAYS_BIRTH'].max()/365, "years old to", 
      -train_main['DAYS_BIRTH'].min()/365, "years old")

Applicants range from 20.517808219178082 years old to 69.12054794520547 years old


In [13]:
# Propose: Bin ages into <= 30 and >30
train_main['Age'] = -train_main['DAYS_BIRTH']/365

def agebin(age):
    """
    Determine if an applicant's age is below or above 30
    arg: age in interest
    return: 1 if below 30, 0 if above
    """
    if age <= 30:
        return 1
    else:
        return 0

train_main['AgeBin'] = train_main['Age'].map(agebin)
train_main['AgeBin'].value_counts()

0    262476
1     45018
Name: AgeBin, dtype: int64

In [14]:
train_main.groupby(['AgeBin'])['TARGET'].mean()
# unprivileged_group: less than 30

AgeBin
0    0.074921
1    0.114576
Name: TARGET, dtype: float64

In [15]:
# Drop original columns
train_main = train_main.drop(columns=['CODE_GENDER','NAME_FAMILY_STATUS','DAYS_BIRTH','Age'])
train_main.shape

(307494, 122)

In [16]:
# read in the ancillary csv's
credit_card = pd.read_csv('cleaned_tables/credit_card_summary.csv')
POS = pd.read_csv('cleaned_tables/df_POS_final2.csv')
install = pd.read_csv('cleaned_tables/df_pay_final2.csv')
bureau = pd.read_csv('cleaned_tables/Bureau_features.csv')
prev = pd.read_csv('cleaned_tables/prevapp_finally_train.csv')

### Get column names of additional tables

In [17]:
cred_col = credit_card.columns
credit_col = cred_col[1:]
len(credit_col) #8 extra

8

In [18]:
POS.drop(['DPD_df_POS','count_df_POS'], axis = 1, inplace = True)

In [19]:
POS_col = POS.columns[1:]
len(POS_col) #4 extra

4

In [20]:
install_col = install.columns[1:]
len(install_col) #12 extra

12

In [21]:
bureau_col = bureau.columns[1:]
len(bureau_col) #15 extra

15

In [22]:
prev_col = prev.columns[1:]
len(prev_col) # 10 extra

10

### merge tables

In [23]:
prev[:5]

,ID_CURR,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY,min_term,max_term,mean_term,DAYS_TERMINATION
0,100002,179055.00,1,179055.0,1.000000,-606,0.0,24.0,24.0,24.0,-17.0
1,100003,1306309.50,3,1452573.0,1.000000,-746,0.0,6.0,12.0,9.0,-527.0
2,100004,24282.00,1,20106.0,1.000000,-815,0.0,4.0,4.0,4.0,-714.0
3,100006,2449829.34,9,1718644.5,0.555556,-181,0.0,0.0,48.0,18.0,0.0
4,100007,903181.50,6,999832.5,1.000000,-374,0.0,18.0,48.0,30.0,0.0


In [24]:
df = train_main.merge(credit_card, how = 'left', on = 'SK_ID_CURR')
df = df.merge(POS, how = 'left', left_on ='SK_ID_CURR', right_on = 'SK_ID_CURR_df_POS').drop('SK_ID_CURR_df_POS', axis = 1)
df = df.merge(install, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR_pay').drop('SK_ID_CURR_pay', axis = 1)
df = df.merge(bureau, how = 'left', on = 'SK_ID_CURR')
df = df.merge(prev, how = 'left', left_on = 'SK_ID_CURR', right_on = 'ID_CURR').drop('ID_CURR',axis = 1)

In [25]:
df[:5]

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,IsMale,MaritalSingle,AgeBin,AVG_CC_BALANCE,MAX_CC_BALANCE,MIN_CC_BALANCE,MED_CC_BALANCE,MAX_CC_DPD,MIN_CC_DPD,AVG_CC_DPD,ZERO_CC_DPD,max_df_POS,min_df_POS,mean_df_POS,percentage_no_DPD_df_POS,NUM_INSTALMENT_NUMBER_pay,DPD_max_pay,DPD_min_pay,DPD_avg_pay,DPD_Count_pay,AMT_INSTALMENT_pay,AMT_PAYMENT_pay,PERCENT_NOT_PAID_pay,OWED_pay,PERCENT_UNDER_PAID_pay,PERCENT_OWED_pay,no_DPD_Percentage_pay,Percent_BU_CREDIT_ACTIVE,COUNT_BU_CREDIT_ACTIVE,AVG_BU_DAYS_CREDIT,MAX_BU_CREDIT_DAY_OVERDUE,MIN_BU_CREDIT_DAY_OVERDUE,AVG_BU_CREDIT_DAY_OVERDUE,percent_BU_CREDIT_DAY_OVERDUE,MAX_BU_DAYS_CREDIT_ENDDATE,MAX_BU_DAYS_ENDDATE,AVG_BU_CNT_CREDIT_PROLONG,MAX_BU_CNT_CREDIT_PROLONG,SUM_BU_AMT_CREDIT_SUM,COUNT_BU_CREDIT_TYPE,MAX_BU_PERCENT_SUM_OVERDUE,AVG_BU_PERCENT_SUM_OVERDUE,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY,min_term,max_term,mean_term,DAYS_TERMINATION
0,100002,1,Cash loans,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,House / apartment,0.018801,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,19.0,-12.0,-31.0,-20.421053,0.0,219625.695,219625.695,0.0,0.000,0.000000,0.000000,1.000000,0.00,4.0,-645.0,0.0,0.0,0.0,0.0,85.0,-36.0,0.0,0.0,315567.0,2.0,0.0,0.0,179055.00,1.0,17905

### Fill nas for the summarized columns
if they did not appear in other tables, they did not have previous applications, credit cards, etc.

In [26]:
addl_col = cred_col.append([POS_col, install_col, bureau_col, prev_col]) # append other tables' columns as well
df[addl_col] = df[addl_col].fillna(0)
df[:5]

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,IsMale,MaritalSingle,AgeBin,AVG_CC_BALANCE,MAX_CC_BALANCE,MIN_CC_BALANCE,MED_CC_BALANCE,MAX_CC_DPD,MIN_CC_DPD,AVG_CC_DPD,ZERO_CC_DPD,max_df_POS,min_df_POS,mean_df_POS,percentage_no_DPD_df_POS,NUM_INSTALMENT_NUMBER_pay,DPD_max_pay,DPD_min_pay,DPD_avg_pay,DPD_Count_pay,AMT_INSTALMENT_pay,AMT_PAYMENT_pay,PERCENT_NOT_PAID_pay,OWED_pay,PERCENT_UNDER_PAID_pay,PERCENT_OWED_pay,no_DPD_Percentage_pay,Percent_BU_CREDIT_ACTIVE,COUNT_BU_CREDIT_ACTIVE,AVG_BU_DAYS_CREDIT,MAX_BU_CREDIT_DAY_OVERDUE,MIN_BU_CREDIT_DAY_OVERDUE,AVG_BU_CREDIT_DAY_OVERDUE,percent_BU_CREDIT_DAY_OVERDUE,MAX_BU_DAYS_CREDIT_ENDDATE,MAX_BU_DAYS_ENDDATE,AVG_BU_CNT_CREDIT_PROLONG,MAX_BU_CNT_CREDIT_PROLONG,SUM_BU_AMT_CREDIT_SUM,COUNT_BU_CREDIT_TYPE,MAX_BU_PERCENT_SUM_OVERDUE,AVG_BU_PERCENT_SUM_OVERDUE,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY,min_term,max_term,mean_term,DAYS_TERMINATION
0,100002,1,Cash loans,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,House / apartment,0.018801,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,19.0,-12.0,-31.0,-20.421053,0.0,219625.695,219625.695,0.0,0.000,0.000000,0.000000,1.000000,0.00,4.0,-645.0,0.0,0.0,0.0,0.0,85.0,-36.0,0.0,0.0,315567.0,2.0,0.0,0.0,179055.00,1.0,17905

### Required preprocessing steps for the whole table
0. Prep table for AIF
1. Split into train/val/test
2. Fill NAs - following the decision made when exploring the main table
3. Set up categorical variables
4. \[potentially\] Rebalance / SMOTE (part of modeling)

### 0. Prep table for AIF

In [27]:
# remove ID column, put MaritalSingle, IsMale, AgeBin, and target to the end because AIF needs sensitive attributes and target
# to be at the end
aif_train = df.drop(columns=['SK_ID_CURR','MaritalSingle','IsMale','AgeBin','TARGET'])
print(aif_train.shape)
aif_train_2 = pd.concat([aif_train, df[['MaritalSingle','IsMale','AgeBin','TARGET']]], axis = 1)
print(aif_train_2.shape)
aif_train_2[:5]

(307494, 166)
(307494, 170)


,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AVG_CC_BALANCE,MAX_CC_BALANCE,MIN_CC_BALANCE,MED_CC_BALANCE,MAX_CC_DPD,MIN_CC_DPD,AVG_CC_DPD,ZERO_CC_DPD,max_df_POS,min_df_POS,mean_df_POS,percentage_no_DPD_df_POS,NUM_INSTALMENT_NUMBER_pay,DPD_max_pay,DPD_min_pay,DPD_avg_pay,DPD_Count_pay,AMT_INSTALMENT_pay,AMT_PAYMENT_pay,PERCENT_NOT_PAID_pay,OWED_pay,PERCENT_UNDER_PAID_pay,PERCENT_OWED_pay,no_DPD_Percentage_pay,Percent_BU_CREDIT_ACTIVE,COUNT_BU_CREDIT_ACTIVE,AVG_BU_DAYS_CREDIT,MAX_BU_CREDIT_DAY_OVERDUE,MIN_BU_CREDIT_DAY_OVERDUE,AVG_BU_CREDIT_DAY_OVERDUE,percent_BU_CREDIT_DAY_OVERDUE,MAX_BU_DAYS_CREDIT_ENDDATE,MAX_BU_DAYS_ENDDATE,AVG_BU_CNT_CREDIT_PROLONG,MAX_BU_CNT_CREDIT_PROLONG,SUM_BU_AMT_CREDIT_SUM,COUNT_BU_CREDIT_TYPE,MAX_BU_PERCENT_SUM_OVERDUE,AVG_BU_PERCENT_SUM_OVERDUE,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY,min_term,max_term,mean_term,DAYS_TERMINATION,MaritalSingle,IsMale,AgeBin,TARGET
0,Cash loans,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,House / apartment,0.018801,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,19.0,-12.0,-31.0,-20.421053,0.0,219625.695,219625.695,0.0,0.000,0.000000,0.000000,1.000000,0.00,4.0,-645.0,0.0,0.0,0.0,0.0,85.0,-36.0,0.0,0.0,315567.0,2.0,0.0,0.0,179055.00,1.0,179055.0,1.000000,-606.0,0.0,24

### 1. Split into train (50%), validate (30%), and test (20%)

In [28]:
from sklearn.model_selection import train_test_split
aif_train, aif_test = train_test_split(aif_train_2, test_size=0.2, random_state=1)
aif_train, aif_val = train_test_split(aif_train, test_size = 0.3, random_state =1)
print(aif_train.shape, aif_val.shape, aif_test.shape)

(172196, 170) (73799, 170) (61499, 170)


### 2. Fill in missing values

In [29]:
# Drop highly correlated "current housing" attributes
aif_train_nona = aif_train.drop(['APARTMENTS_AVG',
'APARTMENTS_MODE',
'BASEMENTAREA_AVG',
'BASEMENTAREA_MODE',
'COMMONAREA_AVG',
'COMMONAREA_MODE',
'ELEVATORS_AVG',
'ELEVATORS_MODE',
'ENTRANCES_AVG',
'ENTRANCES_MODE',
'FLOORSMAX_AVG',
'FLOORSMAX_MODE',
'FLOORSMIN_AVG',
'FLOORSMIN_MODE',
'LANDAREA_AVG',
'LANDAREA_MODE',
'LIVINGAPARTMENTS_AVG',
'LIVINGAPARTMENTS_MODE',
'LIVINGAREA_AVG',
'LIVINGAREA_MODE',
'NONLIVINGAPARTMENTS_AVG',
'NONLIVINGAPARTMENTS_MODE',
'NONLIVINGAREA_AVG',
'NONLIVINGAREA_MODE',
'YEARS_BEGINEXPLUATATION_AVG',
'YEARS_BEGINEXPLUATATION_MEDI',
'YEARS_BEGINEXPLUATATION_MODE',
'YEARS_BUILD_AVG',
'YEARS_BUILD_MODE',
'DEF_60_CNT_SOCIAL_CIRCLE',
'OBS_60_CNT_SOCIAL_CIRCLE'], axis = 1)

aif_val_nona = aif_val.drop(['APARTMENTS_AVG',
'APARTMENTS_MODE',
'BASEMENTAREA_AVG',
'BASEMENTAREA_MODE',
'COMMONAREA_AVG',
'COMMONAREA_MODE',
'ELEVATORS_AVG',
'ELEVATORS_MODE',
'ENTRANCES_AVG',
'ENTRANCES_MODE',
'FLOORSMAX_AVG',
'FLOORSMAX_MODE',
'FLOORSMIN_AVG',
'FLOORSMIN_MODE',
'LANDAREA_AVG',
'LANDAREA_MODE',
'LIVINGAPARTMENTS_AVG',
'LIVINGAPARTMENTS_MODE',
'LIVINGAREA_AVG',
'LIVINGAREA_MODE',
'NONLIVINGAPARTMENTS_AVG',
'NONLIVINGAPARTMENTS_MODE',
'NONLIVINGAREA_AVG',
'NONLIVINGAREA_MODE',
'YEARS_BEGINEXPLUATATION_AVG',
'YEARS_BEGINEXPLUATATION_MEDI',
'YEARS_BEGINEXPLUATATION_MODE',
'YEARS_BUILD_AVG',
'YEARS_BUILD_MODE',
'DEF_60_CNT_SOCIAL_CIRCLE',
'OBS_60_CNT_SOCIAL_CIRCLE'], axis = 1)

aif_test_nona = aif_test.drop(['APARTMENTS_AVG',
'APARTMENTS_MODE',
'BASEMENTAREA_AVG',
'BASEMENTAREA_MODE',
'COMMONAREA_AVG',
'COMMONAREA_MODE',
'ELEVATORS_AVG',
'ELEVATORS_MODE',
'ENTRANCES_AVG',
'ENTRANCES_MODE',
'FLOORSMAX_AVG',
'FLOORSMAX_MODE',
'FLOORSMIN_AVG',
'FLOORSMIN_MODE',
'LANDAREA_AVG',
'LANDAREA_MODE',
'LIVINGAPARTMENTS_AVG',
'LIVINGAPARTMENTS_MODE',
'LIVINGAREA_AVG',
'LIVINGAREA_MODE',
'NONLIVINGAPARTMENTS_AVG',
'NONLIVINGAPARTMENTS_MODE',
'NONLIVINGAREA_AVG',
'NONLIVINGAREA_MODE',
'YEARS_BEGINEXPLUATATION_AVG',
'YEARS_BEGINEXPLUATATION_MEDI',
'YEARS_BEGINEXPLUATATION_MODE',
'YEARS_BUILD_AVG',
'YEARS_BUILD_MODE',
'DEF_60_CNT_SOCIAL_CIRCLE',
'OBS_60_CNT_SOCIAL_CIRCLE'], axis = 1)

In [30]:
print(aif_train_nona.shape, aif_val_nona.shape, aif_test_nona.shape) # 31 columns dropped; 170 --> 139

(172196, 139) (73799, 139) (61499, 139)


In [31]:
# Filling with Medians
columns_with_median = ['COMMONAREA_MEDI',
'LIVINGAPARTMENTS_MEDI',
'FLOORSMIN_MEDI',
'LANDAREA_MEDI',
'BASEMENTAREA_MEDI',
'ELEVATORS_MEDI',
'APARTMENTS_MEDI',
'ENTRANCES_MEDI',
'LIVINGAREA_MEDI',
'FLOORSMAX_MEDI',
'AMT_GOODS_PRICE',
'AMT_ANNUITY']

for col in columns_with_median:
    aif_train_nona[col].fillna(aif_train_nona[col].median(), inplace = True)
    aif_val_nona[col].fillna(aif_train_nona[col].median(), inplace = True)
    aif_test_nona[col].fillna(aif_train_nona[col].median(), inplace = True)

In [32]:
# Filling with Means
columns_with_means = ['YEARS_BUILD_MEDI',
'EXT_SOURCE_1',
'TOTALAREA_MODE',
'EXT_SOURCE_3',
'EXT_SOURCE_2']

for col in columns_with_means:
    aif_train_nona[col].fillna(aif_train_nona[col].mean(), inplace = True)
    aif_val_nona[col].fillna(aif_train_nona[col].mean(), inplace = True)
    aif_test_nona[col].fillna(aif_train_nona[col].mean(), inplace = True)

In [33]:
# Fill with Mode
columns_with_modes = ['FONDKAPREMONT_MODE', 
'HOUSETYPE_MODE',
'EMERGENCYSTATE_MODE']

for col in columns_with_modes:
    aif_train_nona[col].fillna(aif_train_nona[col].mode()[0], inplace = True)
    aif_val_nona[col].fillna(aif_train_nona[col].mode()[0], inplace = True)
    aif_test_nona[col].fillna(aif_train_nona[col].mode()[0], inplace = True)

In [34]:
# Fill with 0's
columns_with_0s = ['NONLIVINGAPARTMENTS_MEDI',
'OWN_CAR_AGE',
'NONLIVINGAREA_MEDI',
'AMT_REQ_CREDIT_BUREAU_YEAR',
'AMT_REQ_CREDIT_BUREAU_DAY',
'AMT_REQ_CREDIT_BUREAU_QRT',
'AMT_REQ_CREDIT_BUREAU_WEEK',
'AMT_REQ_CREDIT_BUREAU_HOUR',
'AMT_REQ_CREDIT_BUREAU_MON',
'OBS_30_CNT_SOCIAL_CIRCLE',
'DEF_30_CNT_SOCIAL_CIRCLE',
'CNT_FAM_MEMBERS',
'DAYS_LAST_PHONE_CHANGE']

for col in columns_with_0s:
    aif_train_nona[col].fillna(0, inplace = True)
    aif_val_nona[col].fillna(0, inplace = True)
    aif_test_nona[col].fillna(0, inplace = True)

In [35]:
# Other special cases
aif_train_nona['WALLSMATERIAL_MODE'].fillna("Others",inplace = True)
aif_train_nona['OCCUPATION_TYPE'].fillna("Unemployed",inplace = True)
aif_train_nona['ORGANIZATION_TYPE'].fillna("Retired",inplace = True)
aif_train_nona['NAME_TYPE_SUITE'].fillna("Unaccompanied",inplace = True)
# aif_train_nona['CODE_GENDER'].fillna("F",inplace = True)

aif_val_nona['WALLSMATERIAL_MODE'].fillna("Others",inplace = True)
aif_val_nona['OCCUPATION_TYPE'].fillna("Unemployed",inplace = True)
aif_val_nona['ORGANIZATION_TYPE'].fillna("Retired",inplace = True)
aif_val_nona['NAME_TYPE_SUITE'].fillna("Unaccompanied",inplace = True)
# aif_val_nona['CODE_GENDER'].fillna("F",inplace = True)

aif_test_nona['WALLSMATERIAL_MODE'].fillna("Others",inplace = True)
aif_test_nona['OCCUPATION_TYPE'].fillna("Unemployed",inplace = True)
aif_test_nona['ORGANIZATION_TYPE'].fillna("Retired",inplace = True)
aif_test_nona['NAME_TYPE_SUITE'].fillna("Unaccompanied",inplace = True)
# no longer need to update gender: X_test_nona['CODE_GENDER'].fillna("F",inplace = True)

In [36]:
# verify that there's no more missing values in the training data
(aif_train_nona.isnull().sum() / len(aif_train_nona)*100).sort_values(ascending = False)

TARGET                           0.0
ELEVATORS_MEDI                   0.0
LIVINGAREA_MEDI                  0.0
LIVINGAPARTMENTS_MEDI            0.0
LANDAREA_MEDI                    0.0
FLOORSMIN_MEDI                   0.0
FLOORSMAX_MEDI                   0.0
ENTRANCES_MEDI                   0.0
COMMONAREA_MEDI                  0.0
LIVE_CITY_NOT_WORK_CITY          0.0
YEARS_BUILD_MEDI                 0.0
BASEMENTAREA_MEDI                0.0
APARTMENTS_MEDI                  0.0
EXT_SOURCE_3                     0.0
EXT_SOURCE_2                     0.0
EXT_SOURCE_1                     0.0
NONLIVINGAPARTMENTS_MEDI         0.0
NONLIVINGAREA_MEDI               0.0
FONDKAPREMONT_MODE               0.0
HOUSETYPE_MODE                   0.0
TOTALAREA_MODE                   0.0
WALLSMATERIAL_MODE               0.0
EMERGENCYSTATE_MODE              0.0
OBS_30_CNT_SOCIAL_CIRCLE         0.0
DEF_30_CNT_SOCIAL_CIRCLE         0.0
DAYS_LAST_PHONE_CHANGE           0.0
FLAG_DOCUMENT_2                  0.0
F

In [37]:
(aif_val_nona.isnull().sum() / len(aif_val_nona)*100).sort_values(ascending = False)

TARGET                           0.0
ELEVATORS_MEDI                   0.0
LIVINGAREA_MEDI                  0.0
LIVINGAPARTMENTS_MEDI            0.0
LANDAREA_MEDI                    0.0
FLOORSMIN_MEDI                   0.0
FLOORSMAX_MEDI                   0.0
ENTRANCES_MEDI                   0.0
COMMONAREA_MEDI                  0.0
LIVE_CITY_NOT_WORK_CITY          0.0
YEARS_BUILD_MEDI                 0.0
BASEMENTAREA_MEDI                0.0
APARTMENTS_MEDI                  0.0
EXT_SOURCE_3                     0.0
EXT_SOURCE_2                     0.0
EXT_SOURCE_1                     0.0
NONLIVINGAPARTMENTS_MEDI         0.0
NONLIVINGAREA_MEDI               0.0
FONDKAPREMONT_MODE               0.0
HOUSETYPE_MODE                   0.0
TOTALAREA_MODE                   0.0
WALLSMATERIAL_MODE               0.0
EMERGENCYSTATE_MODE              0.0
OBS_30_CNT_SOCIAL_CIRCLE         0.0
DEF_30_CNT_SOCIAL_CIRCLE         0.0
DAYS_LAST_PHONE_CHANGE           0.0
FLAG_DOCUMENT_2                  0.0
F

In [38]:
(aif_test_nona.isnull().sum() / len(aif_test_nona)*100).sort_values(ascending = False)

TARGET                           0.0
ELEVATORS_MEDI                   0.0
LIVINGAREA_MEDI                  0.0
LIVINGAPARTMENTS_MEDI            0.0
LANDAREA_MEDI                    0.0
FLOORSMIN_MEDI                   0.0
FLOORSMAX_MEDI                   0.0
ENTRANCES_MEDI                   0.0
COMMONAREA_MEDI                  0.0
LIVE_CITY_NOT_WORK_CITY          0.0
YEARS_BUILD_MEDI                 0.0
BASEMENTAREA_MEDI                0.0
APARTMENTS_MEDI                  0.0
EXT_SOURCE_3                     0.0
EXT_SOURCE_2                     0.0
EXT_SOURCE_1                     0.0
NONLIVINGAPARTMENTS_MEDI         0.0
NONLIVINGAREA_MEDI               0.0
FONDKAPREMONT_MODE               0.0
HOUSETYPE_MODE                   0.0
TOTALAREA_MODE                   0.0
WALLSMATERIAL_MODE               0.0
EMERGENCYSTATE_MODE              0.0
OBS_30_CNT_SOCIAL_CIRCLE         0.0
DEF_30_CNT_SOCIAL_CIRCLE         0.0
DAYS_LAST_PHONE_CHANGE           0.0
FLAG_DOCUMENT_2                  0.0
F

In [39]:
aif_train_nona[:2]

,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AVG_CC_BALANCE,MAX_CC_BALANCE,MIN_CC_BALANCE,MED_CC_BALANCE,MAX_CC_DPD,MIN_CC_DPD,AVG_CC_DPD,ZERO_CC_DPD,max_df_POS,min_df_POS,mean_df_POS,percentage_no_DPD_df_POS,NUM_INSTALMENT_NUMBER_pay,DPD_max_pay,DPD_min_pay,DPD_avg_pay,DPD_Count_pay,AMT_INSTALMENT_pay,AMT_PAYMENT_pay,PERCENT_NOT_PAID_pay,OWED_pay,PERCENT_UNDER_PAID_pay,PERCENT_OWED_pay,no_DPD_Percentage_pay,Percent_BU_CREDIT_ACTIVE,COUNT_BU_CREDIT_ACTIVE,AVG_BU_DAYS_CREDIT,MAX_BU_CREDIT_DAY_OVERDUE,MIN_BU_CREDIT_DAY_OVERDUE,AVG_BU_CREDIT_DAY_OVERDUE,percent_BU_CREDIT_DAY_OVERDUE,MAX_BU_DAYS_CREDIT_ENDDATE,MAX_BU_DAYS_ENDDATE,AVG_BU_CNT_CREDIT_PROLONG,MAX_BU_CNT_CREDIT_PROLONG,SUM_BU_AMT_CREDIT_SUM,COUNT_BU_CREDIT_TYPE,MAX_BU_PERCENT_SUM_OVERDUE,AVG_BU_PERCENT_SUM_OVERDUE,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY,min_term,max_term,mean_term,DAYS_TERMINATION,MaritalSingle,IsMale,AgeBin,TARGET
60955,Cash loans,N,Y,0,225000.0,301500.0,35910.0,301500.0,Unaccompanied,Working,Secondary / secondary special,House / apartment,0.026392,-349,-1397.0,-4787,0.0,1,1,0,1,1,0,Unemployed,2.0,2,2,MONDAY,12,0,1,1,0,1,1,Industry: type 9,0.171084,0.587407,0.510771,0.0802,0.0758,0.756006,0.0207,0.0,0.0345,0.2083,0.2083,0.0482,0.0761,0.0576,0.0,0.0,reg oper account,block of flats,0.0445,"Stone, brick",No,7.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,2.0,0.000000,0.000,0.0,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,37.0,2.0,-63.0,-19.324324,2.0,1403651.025,1403651.025,0.0,0.00,0.000000,0.000000,0.945946,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,4683150.0,8.0,1539000.0,0.375000,-280.0,0.0,12.0,36.0,19.500000,-365.0,0,1,0,1
195516,Cash loans,N,Y,1,67500.0,296505.0,23076.0,247500.0,Unaccompanied,State servant,Secondary / secondary special,House / apartment,0.005144,-4075,-3842.0,-3836,0.0,1,1,0,1,0,0,Core staff,2.0,2,2,TUESDAY,10,0,0,0,0,0,0,Postal,0.502560,0.399408,0.502878,0.0937,0.0997,0.724900,0.0000,0.0,0.2069,0.1667,0.2500,0.0183,0.0770,0.0884,0.0,0.0,reg oper account,block of flats,0.0683,Panel,No,10.0,0.0,-642.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0,70650.641897,141055.245,0.0,49558.95,1.0,0.0,0.045977,0.954023,5.0,0.0,0.073529,0.985294,180.0,15.0,-100.0,-15.372222,5.0,975740.445,1165032.315,0.0,-189291.87,0.011111,-0.193998,0.972222,0.25,4.0,-300.0,0.0,0.0,0.0,0.0,70.0,0.0,0.0,0.0,319235.22,1.0,0.0,0.0,7555

### 3. Identify and codify categorical columns - X_train and X_test

### Identify categorical columns

In [40]:
# Categorical columns with strings
string_categorical = aif_train_nona.columns[aif_train_nona.dtypes == 'object']
s1 = df[string_categorical].nunique()

In [41]:
# string binary
binary_cat = s1[s1 ==2].index 
binary_cat

Index(['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'EMERGENCYSTATE_MODE'],
      dtype='object')

In [42]:
# Categorical columns have numerical values
s2 = aif_train_nona.loc[:,aif_train_nona.dtypes != 'object'].nunique()

In [43]:
# numerical binary: this doesn't really need to be convert! already 0 and 1
# binary_cat = binary_cat.append(s2[s2==2])

In [44]:
#all binary
binary_cat

Index(['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'EMERGENCYSTATE_MODE'],
      dtype='object')

In [45]:
# all non-binary categorical columns
non_bi_cat = s1[s1>2].index # string
non_bi_cat = non_bi_cat.append(s2[(s2>2) & (s2<5)].index) # numerical
non_bi_cat # all

Index(['NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START',
       'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE',
       'WALLSMATERIAL_MODE', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY'],
      dtype='object')

In [46]:
print(len(binary_cat), len(non_bi_cat), len(aif_train_nona.columns))

4 12 139


### Codify categorical columns
- Binary: label encoder
- Non-binary: get_dummies

In [47]:
# binary
for column in binary_cat:
    aif_train_nona[column] = aif_train_nona[column].astype('category').cat.codes
    aif_val_nona[column] = aif_val_nona[column].astype('category').cat.codes
    aif_test_nona[column] = aif_test_nona[column].astype('category').cat.codes

In [48]:
# verifying everyone has two categories
aif_train_nona[binary_cat].nunique()

NAME_CONTRACT_TYPE     2
FLAG_OWN_CAR           2
FLAG_OWN_REALTY        2
EMERGENCYSTATE_MODE    2
dtype: int64

In [49]:
# non-binary
aif_train_nona = pd.get_dummies(aif_train_nona,
                            prefix=non_bi_cat,
                            columns=non_bi_cat)
aif_train_nona.shape #up from 139

(172196, 255)

In [50]:
aif_val_nona = pd.get_dummies(aif_val_nona,
                            prefix=non_bi_cat,
                            columns=non_bi_cat)
aif_val_nona.shape #up from 139

(73799, 255)

In [51]:
aif_test_nona = pd.get_dummies(aif_test_nona,
                            prefix=non_bi_cat,
                            columns=non_bi_cat)
aif_test_nona.shape #up from 139

(61499, 255)

In [52]:
aif_train_nona.columns.symmetric_difference(aif_test_nona.columns) #identical sets of training predictors

Index([], dtype='object')

In [53]:
aif_train_nona.columns.symmetric_difference(aif_val_nona.columns) #identical sets of training predictors

Index([], dtype='object')

#### Check the processed, merged dataset

In [54]:
aif_train_nona.describe()

,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AVG_CC_BALANCE,MAX_CC_BALANCE,MIN_CC_BALANCE,MED_CC_BALANCE,MAX_CC_DPD,MIN_CC_DPD,AVG_CC_DPD,ZERO_CC_DPD,max_df_POS,min_df_POS,mean_df_POS,percentage_no_DPD_df_POS,NUM_INSTALMENT_NUMBER_pay,DPD_max_pay,DPD_min_pay,DPD_avg_pay,DPD_Count_pay,AMT_INSTALMENT_pay,AMT_PAYMENT_pay,PERCENT_NOT_PAID_pay,OWED_pay,PERCENT_UNDER_PAID_pay,PERCENT_OWED_pay,no_DPD_Percentage_pay,Percent_BU_CREDIT_ACTIVE,COUNT_BU_CREDIT_ACTIVE,AVG_BU_DAYS_CREDIT,MAX_BU_CREDIT_DAY_OVERDUE,MIN_BU_CREDIT_DAY_OVERDUE,AVG_BU_CREDIT_DAY_OVERDUE,percent_BU_CREDIT_DAY_OVERDUE,MAX_BU_DAYS_CREDIT_ENDDATE,MAX_BU_DAYS_ENDDATE,AVG_BU_CNT_CREDIT_PROLONG,MAX_BU_CNT_CREDIT_PROLONG,SUM_BU_AMT_CREDIT_SUM,COUNT_BU_CREDIT_TYPE,MAX_BU_PERCENT_SUM_OVERDUE,AVG_BU_PERCENT_SUM_OVERDUE,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY,min_term,max_term,mean_term,DAYS_TERMINATION,MaritalSingle,IsMale,AgeBin,TARGET,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Unemployed,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2

In [55]:
aif_train_nona[:5]

,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AVG_CC_BALANCE,MAX_CC_BALANCE,MIN_CC_BALANCE,MED_CC_BALANCE,MAX_CC_DPD,MIN_CC_DPD,AVG_CC_DPD,ZERO_CC_DPD,max_df_POS,min_df_POS,mean_df_POS,percentage_no_DPD_df_POS,NUM_INSTALMENT_NUMBER_pay,DPD_max_pay,DPD_min_pay,DPD_avg_pay,DPD_Count_pay,AMT_INSTALMENT_pay,AMT_PAYMENT_pay,PERCENT_NOT_PAID_pay,OWED_pay,PERCENT_UNDER_PAID_pay,PERCENT_OWED_pay,no_DPD_Percentage_pay,Percent_BU_CREDIT_ACTIVE,COUNT_BU_CREDIT_ACTIVE,AVG_BU_DAYS_CREDIT,MAX_BU_CREDIT_DAY_OVERDUE,MIN_BU_CREDIT_DAY_OVERDUE,AVG_BU_CREDIT_DAY_OVERDUE,percent_BU_CREDIT_DAY_OVERDUE,MAX_BU_DAYS_CREDIT_ENDDATE,MAX_BU_DAYS_ENDDATE,AVG_BU_CNT_CREDIT_PROLONG,MAX_BU_CNT_CREDIT_PROLONG,SUM_BU_AMT_CREDIT_SUM,COUNT_BU_CREDIT_TYPE,MAX_BU_PERCENT_SUM_OVERDUE,AVG_BU_PERCENT_SUM_OVERDUE,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY,min_term,max_term,mean_term,DAYS_TERMINATION,MaritalSingle,IsMale,AgeBin,TARGET,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Unemployed,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2

In [56]:
# Need to reorder the columns again...
aif_train_ready = aif_train_nona.drop(columns=['MaritalSingle','IsMale','AgeBin','TARGET'])
print(aif_train_ready.shape)
aif_train_ready = pd.concat([aif_train_ready, aif_train_nona[['MaritalSingle','IsMale','AgeBin','TARGET']]], axis = 1)
print(aif_train_ready.shape)
aif_train_ready[:5]

(172196, 251)
(172196, 255)


,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AVG_CC_BALANCE,MAX_CC_BALANCE,MIN_CC_BALANCE,MED_CC_BALANCE,MAX_CC_DPD,MIN_CC_DPD,AVG_CC_DPD,ZERO_CC_DPD,max_df_POS,min_df_POS,mean_df_POS,percentage_no_DPD_df_POS,NUM_INSTALMENT_NUMBER_pay,DPD_max_pay,DPD_min_pay,DPD_avg_pay,DPD_Count_pay,AMT_INSTALMENT_pay,AMT_PAYMENT_pay,PERCENT_NOT_PAID_pay,OWED_pay,PERCENT_UNDER_PAID_pay,PERCENT_OWED_pay,no_DPD_Percentage_pay,Percent_BU_CREDIT_ACTIVE,COUNT_BU_CREDIT_ACTIVE,AVG_BU_DAYS_CREDIT,MAX_BU_CREDIT_DAY_OVERDUE,MIN_BU_CREDIT_DAY_OVERDUE,AVG_BU_CREDIT_DAY_OVERDUE,percent_BU_CREDIT_DAY_OVERDUE,MAX_BU_DAYS_CREDIT_ENDDATE,MAX_BU_DAYS_ENDDATE,AVG_BU_CNT_CREDIT_PROLONG,MAX_BU_CNT_CREDIT_PROLONG,SUM_BU_AMT_CREDIT_SUM,COUNT_BU_CREDIT_TYPE,MAX_BU_PERCENT_SUM_OVERDUE,AVG_BU_PERCENT_SUM_OVERDUE,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY,min_term,max_term,mean_term,DAYS_TERMINATION,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Unemployed,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity 

In [57]:
aif_val_ready = aif_val_nona.drop(columns=['MaritalSingle','IsMale','AgeBin','TARGET'])
aif_val_ready = pd.concat([aif_val_ready, aif_val_nona[['MaritalSingle','IsMale','AgeBin','TARGET']]], axis = 1)

aif_test_ready = aif_test_nona.drop(columns=['MaritalSingle','IsMale','AgeBin','TARGET'])
aif_test_ready = pd.concat([aif_test_ready, aif_test_nona[['MaritalSingle','IsMale','AgeBin','TARGET']]], axis = 1)

## Fairness Modeling

In [65]:
# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.explainers import MetricTextExplainer
from aif360.metrics import ClassificationMetric

In [78]:
# Get DF into IBM format
from aif360 import datasets
aif_train_ready2 = datasets.BinaryLabelDataset(favorable_label = 0.0, unfavorable_label = 1.0, df=aif_train_ready,
                                                      label_names=["TARGET"],
                                                     protected_attribute_names=["AgeBin","IsMale","MaritalSingle"]
                                              privileged_protected_attributes = [0,0,0])

In [80]:
# Disparate impact measurement for gender
metric_aif_train_ready2 = BinaryLabelDatasetMetric(
        aif_train_ready2,
        unprivileged_groups=[{"IsMale":1}], #,{"AgeBin":1},{"MaritalSingle":1}
        privileged_groups=[{"IsMale":0}]) #,{"AgeBin":0},{"MaritalSingle":0}
explainer_aif_train_ready2 = MetricTextExplainer(metric_aif_train_ready2)

print(explainer_aif_train_ready2.disparate_impact())

Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.9663256523934279


In [81]:
# Disparate impact measurement for age
metric_aif_train_ready2 = BinaryLabelDatasetMetric(
        aif_train_ready2,
        unprivileged_groups=[{"AgeBin":1}],
        privileged_groups=[{"AgeBin":0}])
explainer_aif_train_ready2 = MetricTextExplainer(metric_aif_train_ready2)

print(explainer_aif_train_ready2.disparate_impact())

Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.957318541834988


In [82]:
# Disparate impact measurement for Marrige Status
metric_aif_train_ready2 = BinaryLabelDatasetMetric(
        aif_train_ready2,
        unprivileged_groups=[{"MaritalSingle":1}],
        privileged_groups=[{"MaritalSingle":0}])
explainer_aif_train_ready2 = MetricTextExplainer(metric_aif_train_ready2)

print(explainer_aif_train_ready2.disparate_impact())

Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.9924151064756674


#### Comments:
Based on the initial measures of disparate impact, it does not appear we have serious concerns on fairness for the three protected attributes. As such, our modeling efforts for explainability were most likely based on a fair training set. Nonetheless, we trained a fair classifier with IBM's PrejudiceRemover for demonstration below.

**Note**: According to the source code of [PrejudiceRemover]( https://github.com/IBM/AIF360/blob/master/aif360/algorithms/inprocessing/prejudice_remover.py), the code only works for one protected class. Therefore, we chose to use Age as the protected attribute since it was one of the most important predictors in RF.

In [59]:
from aif360.algorithms.inprocessing import PrejudiceRemover

In [202]:
from collections import defaultdict
def test(dataset, model, thresh_arr):
    try:
        # sklearn classifier
        y_val_pred_prob = model.predict_proba(dataset.features)
        pos_ind = np.where(model.classes_ == dataset.favorable_label)[0][0]
    except AttributeError:
        # aif360 inprocessing algorithm
        y_val_pred_prob = model.predict(dataset).scores
        pos_ind = 0
    metric_arrs = defaultdict(list)
    for thresh in thresh_arr:
        y_val_pred = (y_val_pred_prob[:, pos_ind] > thresh).astype(np.float64)
        dataset_pred = dataset.copy()
        dataset_pred.labels = y_val_pred
        metric = ClassificationMetric(
                dataset, dataset_pred,
                unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
        metric_arrs['bal_acc'].append((metric.true_positive_rate()
                                     + metric.true_negative_rate()) / 2)
        metric_arrs['avg_odds_diff'].append(metric.average_odds_difference())
        metric_arrs['disp_imp'].append(metric.disparate_impact())
        metric_arrs['stat_par_diff'].append(metric.statistical_parity_difference())
        metric_arrs['eq_opp_diff'].append(metric.equal_opportunity_difference())
    return metric_arrs

In [205]:
def describe_metrics(metrics, thresh_arr):
    best_ind = np.argmax(metrics['bal_acc'])
    print("Threshold corresponding to Best balanced accuracy: {:6.4f}".format(thresh_arr[best_ind]))
    print("Best balanced accuracy: {:6.4f}".format(metrics['bal_acc'][best_ind]))
#     disp_imp_at_best_ind = np.abs(1 - np.array(metrics['disp_imp']))[best_ind]
    disp_imp_at_best_ind = 1 - min(metrics['disp_imp'][best_ind], 1/metrics['disp_imp'][best_ind])
    print("Corresponding 1-min(DI, 1/DI) value: {:6.4f}".format(disp_imp_at_best_ind))
    print("Corresponding average odds difference value: {:6.4f}".format(metrics['avg_odds_diff'][best_ind]))
    print("Corresponding statistical parity difference value: {:6.4f}".format(metrics['stat_par_diff'][best_ind]))
    print("Corresponding equal opportunity difference value: {:6.4f}".format(metrics['eq_opp_diff'][best_ind]))

In [61]:
aif_train_ready2 = aif360.datasets.BinaryLabelDataset(favorable_label=0.0, unfavorable_label=1.0, df=aif_train_ready, 
                                                      label_names=["TARGET"], 
                                                      protected_attribute_names=["AgeBin"], #,"MaritalSingle","IsMale"
                                                      privileged_protected_attributes = [0]) # 0 = <30, single, male

In [62]:
model = PrejudiceRemover(eta=25.0)
prmodel=model.fit(aif_train_ready2)

In [206]:
unprivileged_groups = [{"AgeBin": 1}]
privileged_groups = [{"AgeBin": 0}]
thresh_arr = np.linspace(0.01, 0.50, 50)
dataset = aif_val_ready.copy()
dataset = aif360.datasets.BinaryLabelDataset(favorable_label=0.0, unfavorable_label=1.0, df=dataset, 
                                                      label_names=["TARGET"], 
                                                      protected_attribute_names=["AgeBin"], #,"MaritalSingle","IsMale"
                                                      privileged_protected_attributes = [0]) # 0 = <30, single, male
val_metrics = test(dataset=dataset,
                   model=prmodel,
                   thresh_arr=thresh_arr)
pr_orig_best_ind = np.argmax(val_metrics['bal_acc'])

In [207]:
describe_metrics(val_metrics, thresh_arr)

Threshold corresponding to Best balanced accuracy: 0.5000
Best balanced accuracy: 0.5000
Corresponding 1-min(DI, 1/DI) value: 0.3675
Corresponding average odds difference value: -0.0000
Corresponding statistical parity difference value: 0.0001
Corresponding equal opportunity difference value: 0.0001


In [208]:
dataset = aif_test_ready.copy()
dataset = aif360.datasets.BinaryLabelDataset(favorable_label=0.0, unfavorable_label=1.0, df=dataset, 
                                                      label_names=["TARGET"], 
                                                      protected_attribute_names=["AgeBin"], #,"MaritalSingle","IsMale"
                                                      privileged_protected_attributes = [0]) # 0 = <30, single, male
test_metrics = test(dataset=dataset,
                   model=prmodel,
                   thresh_arr=[thresh_arr[pr_orig_best_ind]])
pr_orig_best_ind = np.argmax(val_metrics['bal_acc'])

In [209]:
describe_metrics(test_metrics, [thresh_arr[pr_orig_best_ind]])

Threshold corresponding to Best balanced accuracy: 0.5000
Best balanced accuracy: 0.5001
Corresponding 1-min(DI, 1/DI) value: 0.5715
Corresponding average odds difference value: 0.0001
Corresponding statistical parity difference value: 0.0001
Corresponding equal opportunity difference value: 0.0001


**As can be seen from both the validation and test results, even given the default threshold of 0.5, most fairness metrics are already near perfect.**